In [1]:
from __future__ import absolute_import, division, print_function
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
x = tf.ones((2, 2))

with tf.GradientTape() as tape:
  tape.watch(x)
  y = tf.reduce_sum(x)
  z = tf.multiply(y, y)

In [0]:
# Derivative of z with repect to the original input tensor x
dz_dx = tape.gradient(z, x)
for i in [0, 1]:
  for j in [0, 1]:
    assert dz_dx[i][j] == 8.0

In [0]:
# Can also request gradients of the output with respect to
# intermediate values computed during "recorded" tf.GradientTape context
x = tf.ones((2, 2))

with tf.GradientTape() as tape:
  tape.watch(x)
  y = tf.reduce_sum(x)
  z = tf.multiply(y, y)

# Use the tape to compute the derivatives of z with respect to the
# intermediate value y.
dz_dy = tape.gradient(z, y)
assert dz_dy == 8.0

In [0]:
# By default, the resources held by a GradientTape are released
# as soon as GradientTape.gradient() method is called.
# To compute multiple gradients over the same computation,
# create a `persistent` gradient tape.
# This allows multiple calls to the `gradient()` method as
# resources are released when the tape object is garbeage collected
x = tf.constant(3, dtype=tf.float32)
with tf.GradientTape(persistent=True) as tape:
  tape.watch(x)
  y = x*x
  z = y*y
dz_dx = tape.gradient(z, x)
dy_dx = tape.gradient(y, x)
del tape # Drop the reference to the tape

In [0]:
# Recoding control flow
def f(x, y):
  output = 1.0
  for i in range(y):
    if i > 1 and i < 5:
      output = tf.multiply(output, x)
  return output

def grad(x, y):
  with tf.GradientTape() as t:
    t.watch(x)
    out = f(x, y)
  return t.gradient(out, x)

x = tf.convert_to_tensor(2.0)

assert grad(x, 6).numpy() == 12.0
assert grad(x, 5).numpy() == 12.0
assert grad(x, 4).numpy() == 4.0

In [0]:
# gradient computation is recorded as well.
x = tf.Variable(1.0)

with tf.GradientTape() as t:
  with tf.GradientTape() as t2:
    y = x * x * x
  # Compute the gradient the 't' context manager
  # which means the gradient computation is differentiable as well.
  dy_dx = t2.gradient(y, x)
d2y_dx2 = t.gradient(dy_dx, x)

assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0